# Lab 5: Explore Kafka Streaming (Google Colab Version)

In this Colab version we:

- Download and run **Apache Kafka** directly inside the Colab environment  
- Use Kafka’s command-line tools (`kafka-topics.sh`, `kafka-console-producer.sh`, `kafka-console-consumer.sh`)  
- Script the messages instead of typing everything manually, so the lab is fully reproducible  

> 💡 **Goal:** Experience how Apache Kafka handles real-time data streaming, topics, producers, consumers, and fault tolerance — all from a single Colab notebook.


## Task 1 – Setting up a Kafka Cluster


In **Colab** we:

1. Install Java (required for Kafka)  
2. Download and extract a Kafka binary distribution  
3. Start **Zookeeper** and **Kafka** as background processes inside the Colab VM  

Run the following cell **once** at the start of the lab to set up Kafka.


In [1]:
# Task 1 – Install Java, download Kafka, and start Zookeeper + Kafka
# This may take a few minutes the first time it runs.

import os, time

# Install Java (needed by Kafka)
!apt-get -y update
!apt-get -y install openjdk-11-jre-headless

# Download Kafka (you can change the version if needed)
if not os.path.exists("kafka"):
    !wget -q https://downloads.apache.org/kafka/3.7.0/kafka_2.13-3.7.0.tgz
    !tar -xzf kafka_2.13-3.7.0.tgz
    !mv kafka_2.13-3.7.0 kafka

# Start Zookeeper (background/daemon)
!kafka/bin/zookeeper-server-start.sh -daemon kafka/config/zookeeper.properties
print("Starting Zookeeper...")
time.sleep(10)  # give Zookeeper time to start

# Start Kafka broker (background/daemon)
!kafka/bin/kafka-server-start.sh -daemon kafka/config/server.properties
print("Starting Kafka broker...")
time.sleep(15)  # give Kafka time to start

print("Kafka cluster should now be running on localhost:9092.")

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 https://cli.github.com/packages stable InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.6 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,153 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,491 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa

You can optionally check that the Kafka processes are running (they should contain `zookeeper` and `kafka.Kafka`):

In [4]:
# Optional: Check running Java processes (may show zookeeper + kafka)
!ps aux | grep -E "zookeeper|kafka.Kafka" | head

root        1662  0.0  0.0   7376  3536 ?        S    18:05   0:00 /bin/bash -c ps aux | grep -E "zookeeper|kafka.Kafka" | head
root        1664  0.0  0.0   6624  2388 ?        S    18:05   0:00 grep -E zookeeper|kafka.Kafka


## Task 2 – Creating a Kafka Topic

In the original lab, you opened a shell inside the Kafka container and ran:

```bash
kafka-topics --create --topic streaming-demo --bootstrap-server localhost:9092 --partitions 3 --replication-factor 1
kafka-topics --list --bootstrap-server localhost:9092
```

In this Colab version, we use the same Kafka CLI tools, but with the `.sh` suffix and the local `kafka` directory.

We will:

1. Create a topic named **`streaming-demo`** with 3 partitions and replication factor 1  
2. List all topics to confirm it was created successfully  


In [5]:
# Task 2 – Create the 'streaming-demo' topic
!kafka/bin/kafka-topics.sh --create   --topic streaming-demo   --bootstrap-server localhost:9092   --partitions 3   --replication-factor 1

# Verify that the topic exists
!kafka/bin/kafka-topics.sh --list --bootstrap-server localhost:9092

/bin/bash: line 1: kafka/bin/kafka-topics.sh: No such file or directory
/bin/bash: line 1: kafka/bin/kafka-topics.sh: No such file or directory


## Task 3 – Creating a Kafka Producer (Message Publisher)

In the original lab, you ran a producer inside the Kafka container and **typed messages manually**:

```bash
kafka-console-producer --topic streaming-demo --bootstrap-server localhost:9092

Hello Kafka!
Streaming data test.
Another real-time message.
```

In this notebook, we’ll do the same thing but **script the messages** so the lab is reproducible without manual typing.

We will:

- Use `kafka-console-producer.sh` to publish a few test messages to the `streaming-demo` topic  
- Treat these messages as our **synthetic streaming data**


In [6]:
# Task 3 – Send some synthetic messages to the 'streaming-demo' topic

messages = """Hello Kafka!
Streaming data test.
Another real-time message.
Synthetic message 4.
Synthetic message 5.
"""

print("Sending messages to Kafka:\n", messages)

# Pipe the messages into the Kafka console producer
# The producer will read from stdin and send each line as a message
get_ipython().system('printf "%s" "{messages}" | kafka/bin/kafka-console-producer.sh --topic streaming-demo --bootstrap-server localhost:9092')

Sending messages to Kafka:
 Hello Kafka!
Streaming data test.
Another real-time message.
Synthetic message 4.
Synthetic message 5.

/bin/bash: line 6: kafka/bin/kafka-console-producer.sh: No such file or directory


## Task 4 – Creating a Kafka Consumer (Message Subscriber)

In the original lab, you:

1. Opened a second PowerShell window  
2. Opened a bash shell inside the Kafka container  
3. Ran a consumer from the beginning of the topic and watched the messages appear in real time:

```bash
kafka-console-consumer --topic streaming-demo --bootstrap-server localhost:9092 --from-beginning
```

In this Colab notebook, we’ll do the same thing, but we’ll add a **timeout** so the consumer doesn’t run forever.

We will:

- Start a consumer with `--from-beginning` so it reads **all** messages from the topic  
- Use `--timeout-ms` so the command exits after a short period of inactivity  


In [7]:
# Task 4 – Consume messages from the 'streaming-demo' topic

print("Consuming messages (from beginning)...")
# --timeout-ms 5000 means: stop after 5 seconds with no new messages
!kafka/bin/kafka-console-consumer.sh --topic streaming-demo --bootstrap-server localhost:9092 --from-beginning --timeout-ms 5000

Consuming messages (from beginning)...
/bin/bash: line 1: kafka/bin/kafka-console-consumer.sh: No such file or directory


### 4.1 Add More Messages and Confirm They Appear

In the original lab, you then typed additional lines in the producer window and verified that they showed up in the consumer.

Here, we will:

1. Send a few more synthetic messages  
2. Run the consumer again from the beginning to see **all** messages in the topic (old + new)


In [9]:
# Send a few additional messages
more_messages = """New message after initial batch.
Real-time update 1.
Real-time update 2.
"""

print("Sending additional messages to Kafka:\n", more_messages)
get_ipython().system('printf "%s" "{more_messages}" | kafka/bin/kafka-console-producer.sh --topic streaming-demo --bootstrap-server localhost:9092')

print("\nRe-consuming messages from the beginning (should include ALL messages so far):")
!kafka/bin/kafka-console-consumer.sh --topic streaming-demo --bootstrap-server localhost:9092 --from-beginning --timeout-ms 5000

Sending additional messages to Kafka:
 New message after initial batch.
Real-time update 1.
Real-time update 2.

/bin/bash: line 4: kafka/bin/kafka-console-producer.sh: No such file or directory

Re-consuming messages from the beginning (should include ALL messages so far):
/bin/bash: line 1: kafka/bin/kafka-console-consumer.sh: No such file or directory


## Task 5 – Demonstrating Kafka Fault Tolerance and Scalability


In this Colab we perform the following:

- **Stopping the Kafka process** using `pkill`  
- Trying to send new messages (this should fail or hang)  
- Restarting the Kafka broker  
- Re-consuming from the beginning to show that previously written messages are still available

> ⚠️ Note: In a production cluster, **fault tolerance** comes from having multiple brokers and a replication factor > 1.  
> Here we are just demonstrating that **restarting the broker** does not lose already committed messages.


### 5.1 Stop the Kafka Broker

We will stop the Kafka broker process but leave Zookeeper running.


In [10]:
# Attempt to stop the Kafka broker process
print("Stopping Kafka broker...")
!pkill -f kafka.Kafka || echo "Kafka process not found (may already be stopped)."

# Check processes to confirm
!ps aux | grep -E "zookeeper|kafka.Kafka" | head

Stopping Kafka broker...
^C
root        1998  0.0  0.0   7376  3456 ?        S    18:07   0:00 /bin/bash -c ps aux | grep -E "zookeeper|kafka.Kafka" | head
root        2000  0.0  0.0   6624  2332 ?        S    18:07   0:00 grep -E zookeeper|kafka.Kafka


### 5.2 Try Sending Messages While Broker is Down

Now we attempt to send additional messages with the producer.  
This should **fail** or result in errors, because the Kafka broker is not running.


In [11]:
failed_messages = """This message should fail 1.
This message should fail 2.
"""

print("Trying to send messages while Kafka broker is stopped (expect errors)...")
get_ipython().system('printf "%s" "{failed_messages}" | kafka/bin/kafka-console-producer.sh --topic streaming-demo --bootstrap-server localhost:9092 || echo "Producer failed as expected."')

Trying to send messages while Kafka broker is stopped (expect errors)...
/bin/bash: line 3: kafka/bin/kafka-console-producer.sh: No such file or directory
Producer failed as expected.


### 5.3 Restart Kafka Broker and Verify Message Retention

Now we restart the Kafka broker and run the consumer again from the beginning.

You should see that **all messages sent before the broker was stopped** are still present.


In [12]:
import time

print("Restarting Kafka broker...")
!kafka/bin/kafka-server-start.sh -daemon kafka/config/server.properties
time.sleep(15)

print("Kafka broker restarted. Re-consuming from the beginning to verify retained messages:")
!kafka/bin/kafka-console-consumer.sh --topic streaming-demo --bootstrap-server localhost:9092 --from-beginning --timeout-ms 5000

Restarting Kafka broker...
/bin/bash: line 1: kafka/bin/kafka-server-start.sh: No such file or directory
Kafka broker restarted. Re-consuming from the beginning to verify retained messages:
/bin/bash: line 1: kafka/bin/kafka-console-consumer.sh: No such file or directory


## Lab Review

1. **What happens if a Kafka broker fails? How does Kafka ensure fault tolerance?**  
   - Think about **replication** and **partitions** across multiple brokers in a production cluster.  

2. **Why does Kafka use partitions, and how does that improve scalability?**  
   - A. It allows parallel processing across consumers  
   - B. It ensures message ordering *within* partitions  
   - C. Both A and B  

In this Colab lab, you:
- Set up a **single-node Kafka cluster** inside the notebook  
- Created a **topic** with multiple partitions  
- Used a **producer** to send streaming messages (synthetic data)  
- Used a **consumer** to read messages from the beginning  
- Simulated a **broker failure** and verified that previously committed messages were still available after restart  

> ✅ You have successfully completed the Kafka streaming lab in Google Colab!


### (Optional) Cleanup

If you want to stop Kafka and Zookeeper at the end of the lab, you can run the following cell.


In [13]:
# Optional cleanup: stop Kafka and Zookeeper
print("Stopping Kafka and Zookeeper (optional cleanup)...")
!pkill -f kafka.Kafka || echo "Kafka broker already stopped."
!pkill -f zookeeper || echo "Zookeeper already stopped."
!ps aux | grep -E "zookeeper|kafka.Kafka" | head

Stopping Kafka and Zookeeper (optional cleanup)...
^C
^C
root        2191  0.0  0.0   7376  3496 ?        S    18:07   0:00 /bin/bash -c ps aux | grep -E "zookeeper|kafka.Kafka" | head
root        2193  0.0  0.0   6624  2568 ?        S    18:07   0:00 grep -E zookeeper|kafka.Kafka
